# Protein Mutation Tutorial for $\lambda$ Dynamics Workshop

In [ ]:
import alf
import os, subprocess, shutil
import numpy as np
alf_tutorial_dir=os.getcwd()

## I. Introduction

### I.A. Free Energy Cycle

Today we will be evaluating relative protein folding free energies (i.e. relative stabilities) by considering the free energy of mutation in the folded ensemble and in the unfolded ensemble.

Other free energies, such as relative protein-protein binding free energies can be obtained with these same techniques by evaluating the mutations in different ensembles.

<img src="Figures/CycleBoth.jpg" alf="cycle" width=400/>

### I.B. Approximations

The unfolded ensemble is structurally diverse and relaxes slowly. To avoid sampling this difficult ensemble, we note that amino acids are extended and solvent exposed in the unfolded ensemble, and approximate the unfolded ensemble with a short pentapeptide. The mutating amino acid is flanked by two non-mutating residues on each side.

While a full protein is typically capped by charged caps under normal pH conditions, the ends of this peptide are artificially truncated, so we cap them with neutral groups (ACE ad CT3 in CHARMM) to avoid perturbing local electrostatics.

When multiple mutation sites are considered, we approximate their effects as being additive in the unfolded ensemble, unless they are adjacent or next to adjacent in sequence. Nearby residues such as these are modeled in the same peptide with two non-mutating residues on each side.

### I.C. Patches

For small molecule perturbations, one must go through the process of generating new patches for each new systems because the ligands and perturbations of interest are always unique. Protein mutations are more convenient in this regard. Protein mutations are always the same, so the set of patches may be generated once and applied to any system.

#### I.C.1. Side Chain Patches

The first study amino acid mutations with $\lambda$ dynamics (in T4 lysozyme) placed the protein backbone in the environment (with a single C$\alpha$ atom) and used patches for the side chains (which gives many C$\beta$ atoms). An immediately obvious limitation of this is that glycine and proline mutations cannot be treated with this approach because backbone parameters and connectivity change.

This approach works because the alchemical regions are connected to the environment in just one place, via the C$\alpha$-C$\beta$ bond, and this allows the contribution of the unscaled bonded interactions to be factored out of the partition function by a change of variables when $\lambda=0$. (There is a caveat here about angles.) As the free energy is proportional to the log of the partition function, their contribution to $\Delta G$ is the same in both ensembles and cancels out.

T4 Lysozyme Citation: DOI: <a href="https://doi.org/10.1002/pro.3500">10.1002/pro.3500</a>

#### I.C.2. Whole Residue Patches for Proline and Glycine

An obvious limitation of the side chain perturbation scheme is that backbone parameters and connectivity change for glycine and especially for proline mutations, so they cannot be treated with the side chain perturbation scheme. An obvious alternative is to scale the entire residue by $\lambda$. However, the alchemical group is now bonded to the rest of the system in two places, and unscaled bonds can no longer be factored out of the partition function when $\lambda=0$. Consider an example of a problem this causes: in mutating from proline to glycine, when proline is off, the ring topology still restricts rotation around the protein $\phi$ angle, so glycine loses entropy it ought to have in the unfolded ensemble, and the $\Delta\Delta G$ favors glycine more than it should, because the entropic cost of folding glycine is artificially reduced.

In order to scale the entire residue by $\lambda$ two additional techniques were developed. The first was a soft bond scaled by $\lambda$ in the proline side chain. This allows free rotation around the backbone $\phi$ angle when $\lambda=0$, and while the whole residue cannot be factored out of the partition function, at least the side chains and HN atoms can be.

The second development was to scale all the remaining backbone bonds which could not be factored out of the partition function by $\lambda$ so they do not contribute to the partition function at $\lambda=0$, and harmonically restrain these gas phase atoms to an analogous atom on another substituent so they do not drift off. These harmonic restraints can also be factored out of the partition function.

These two developments lead to two additional sections in the BLOCK module. The CATS section (Constrained ATom Scaling) lists groups of atoms that are restrained together whose bonded interactions are scaled by $\lambda$. There must be one atom from each substituent in each CATS selection. The SOBO section (SOft BOnds) list pairs of bonded atoms whose bond term (and associated angle terms) should be scaled by $\lambda$.

An additional complication of this approach is that adjacent mutations must have bonded terms added between mutated forms of the residues, necessitating a more complicated patching procedure. Fortunately, this patching is scripted.

DOI: <a href="https://doi.org/10.1002/jcc.26525">10.1002/jcc.26525</a>

## II. CHARMM-GUI Setup

Today's tutorial will focus on the three site mutant in T4 lysozyme (PDB: 1L63). The "native" sequence in this case already posesses two mutations to remove the disulfide bond. Within this background we will consider combinations of three mutations: I17M, I27M, and L33M. Experimental data exists for 6 of the 8 combinations:

```
17 27 33  ddG
 I  I  L  0.0
 I  I  M  2.0
 I  M  L  3.1
 I  M  M  3.05
 M  I  L  2.2
 M  I  M  ???
 M  M  L  ???
 M  M  M  3.3
```

Salt conditions are 100 mM NaCl, and temperature was 25$^{\circ}$ C.

### II.A. Patches

Experiments were performed at a pH of 5.4. ProPKA indicates no D or E residues are protonated at this pKa, but that the H is in the protonated state. There are no disulfide bonds to include. This means no patches need to be applied on CHARMM-GUI, but HIS31 should be "mutated" to HSP.

ProPKA may be downloaded and run through python, or can be run using CRIMM as a wrapper when one needs to determine protonation states in a new protein of interest. Disulfides can be extracted from the PDB header, by inspection, or using CRIMM.

The full sequence should have NTER and CTER caps; the peptides should each have ACE and CT3 caps.

Generate the folded structure and the three peptides on CHARMM-GUI. Note that you only need to generate the native sequence. Patches for alchemical mutations will be added later.

<a href="https://www.charmm-gui.org/?doc=input/solution">CHARMM-GUI</a>

## III. Setting Up prep Directories

## IV. Running ALF

In [ ]:
os.environ['SLURMOPTSMD']='--time=240 --ntasks=1 --tasks-per-node=1 --cpus-per-task=1 -p gpu -A rhayes1_lab_gpu --gres=gpu:1 --export=ALL'
os.environ['SLURMOPTSPP']='--time=240 --ntasks=1 --tasks-per-node=1 --cpus-per-task=1 -p gpu -A rhayes1_lab_gpu --gres=gpu:1 --export=ALL'

In [ ]:
os.chdir(alf_tutorial_dir+'/t4l_fold')
!./subsetAll.sh
os.chdir(alf_tutorial_dir)

## V. Analyzing Results

Apply independent peptide approximation to get the free energy of the unfolded ensemble.

In [7]:
!echo I17M
!cat prerun/t4l_i17m/Result.txt
!echo I27M
!cat prerun/t4l_i27m/Result.txt
!echo L33M
!cat prerun/t4l_l33m/Result.txt

import math

fpin=[]
fpin.append(open("prerun/t4l_i17m/Result.txt","r"))
fpin.append(open("prerun/t4l_i27m/Result.txt","r"))
fpin.append(open("prerun/t4l_l33m/Result.txt","r"))
fpout=open("prerun/ResultU.txt","w")

i1=[]
V=[]
E=[]
indices=[]
for i in range(0,len(fpin)):
  i1.append([])
  V.append([])
  E.append([])
  indices.append(0)
  lines=fpin[i].readlines()
  for j in range(0,len(lines)):
    line=lines[j].split()

    i1[i].append(int(line[0]))
    V[i].append(float(line[1]))
    E[i].append(float(line[3]))

while indices[0]<len(i1[0]):
  Vi=0
  Ei=0
  for i in range(0,len(fpin)):
    fpout.write("%2d " % (i1[i][indices[i]],))
    Vi+=V[i][indices[i]]
    Ei=math.sqrt(Ei**2 + E[i][indices[i]]**2)
  fpout.write("%8.3f +/- %5.3f\n" % (Vi,Ei))
  for i in range(0,len(fpin)):
    indices[len(fpin)-1-i]+=1
    if indices[len(fpin)-1-i]==len(i1[len(fpin)-1-i]) and (len(fpin)-1-i)!=0:
      indices[len(fpin)-1-i]=0
    else:
      break

for i in range(0,len(fpin)):
  fpin[i].close()
fpout.close()

!echo Unfolded ensemble
!cat prerun/ResultU.txt

I17M
 0    0.000 +/- 0.028
 1   -6.606 +/- 0.071
I27M
 0    0.000 +/- 0.010
 1   -6.507 +/- 0.015
L33M
 0    0.000 +/- 0.012
 1   12.225 +/- 0.022
Unfolded ensemble
 0  0  0    0.000 +/- 0.032
 0  0  1   12.225 +/- 0.037
 0  1  0   -6.507 +/- 0.034
 0  1  1    5.718 +/- 0.039
 1  0  0   -6.606 +/- 0.073
 1  0  1    5.619 +/- 0.075
 1  1  0  -13.113 +/- 0.074
 1  1  1   -0.888 +/- 0.076


In [8]:
!echo folded
!cat prerun/t4l_fold/Result.txt

import math

fp1=open("prerun/ResultU.txt","r")
fp2=open("prerun/t4l_fold/Result.txt","r")
fp3=open("prerun/Result.txt","w")

lines1=fp1.readlines()
lines2=fp2.readlines()

nsites=len(lines1[0].split())-3
for i in range(0,len(lines1)):
  line1=lines1[i].split()
  line2=lines2[i].split()

  i1=[]
  for j in range(0,nsites):
    i1.append(int(line1[j]))
  V=float(line2[nsites])-float(line1[nsites])
  E=math.sqrt(float(line2[nsites+2])**2 + float(line1[nsites+2])**2)

  for j in range(0,nsites):
    fp3.write("%2d " % (i1[j],))
  fp3.write("%8.3f +/- %5.3f\n" % (V,E))

fp1.close()
fp2.close()
fp3.close()

!echo ddG
!cat prerun/Result.txt

folded
 0  0  0    0.000 +/- 0.043
 0  0  1   14.870 +/- 0.092
 0  1  0   -2.598 +/- 0.026
 0  1  1   11.691 +/- 0.052
 1  0  0   -5.591 +/- 0.026
 1  0  1    7.983 +/- 0.096
 1  1  0   -8.714 +/- 0.051
 1  1  1    4.171 +/- 0.087
ddG
 0  0  0    0.000 +/- 0.054
 0  0  1    2.645 +/- 0.099
 0  1  0    3.909 +/- 0.043
 0  1  1    5.973 +/- 0.065
 1  0  0    1.015 +/- 0.077
 1  0  1    2.364 +/- 0.122
 1  1  0    4.399 +/- 0.090
 1  1  1    5.059 +/- 0.116


At this point, one could consider computing Pearson's correlation and root mean squared error with the experimental values

```
17 27 33  ddG
 I  I  L  0.0
 I  I  M  2.0
 I  M  L  3.1
 I  M  M  3.05
 M  I  L  2.2
 M  I  M  ???
 M  M  L  ???
 M  M  M  3.3
```

## VI. Exercises

* Set up L33M peptide on CHARMM-GUI
* Run L33M peptide with ALF